In [ ]:
import pandas as pd
import numpy as np
import re 
import random
import math
import pickle
import gc
from tqdm.notebook import tqdm
import os
import glob
import warnings
from numpy import dot
from numpy.linalg import norm

warnings.filterwarnings(action='ignore')

In [ ]:
epochs = 150
batch_size = 32

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)

### Data Loading

In [ ]:
# tcae concat
tcae = pd.read_csv('TJAE/tcae_charlie_embedding.csv')
tcae_vocal = pd.read_csv('TJAE/tjae_vocal.csv')
tcae.index = pd.read_csv('TJAE/all.csv')['vocal']
tcae_vocal = tcae_vocal.drop('Unnamed: 0',axis=1)
tcae_vocal.index = ['보컬보컬']
tcae_all = pd.concat([tcae,tcae_vocal])

# nn np concat
df = pd.read_csv('separate/extract_neg_neg.csv',index_col=0)
df2 = pd.read_csv('separate/extract_neg_pos.csv',index_col=0)
df3 = pd.read_csv('separate/extract_pitch_neg_neg.csv',index_col=0)
df4 = pd.read_csv('separate/extract_pitch_neg_pos.csv',index_col=0)

df.columns = ['보컬보컬']
df2.columns = ['보컬보컬']
df3.columns = ['보컬보컬']
df4.columns = ['보컬보컬']
  
nn_a = pd.read_csv('extract_model/nn.csv')
np_a = pd.read_csv('extract_model/np.csv')

nn_all = pd.concat([nn_a,df],axis=1).T
np_all = pd.concat([np_a,df2],axis=1).T

nn_pitch_all = pd.concat([nn_a,df3],axis=1).T
np_pitch_all = pd.concat([np_a,df4],axis=1).T

### 차원축소 및 유사도(코사인,피어슨)을 활용하여 np,pn,nn 테스트 

In [ ]:
# !pip install umap-learn

In [ ]:
import umap

In [ ]:
#차원축소 - UMAP
reducer = umap.UMAP(n_components=30,random_state=42)

tcae_embedding = reducer.fit_transform(tcae_all)
nn_embedding = reducer.fit_transform(nn_all)
np_embedding = reducer.fit_transform(np_all)
nn_pit_embedding = reducer.fit_transform(nn_pitch_all)
np_pit_embedding = reducer.fit_transform(np_pitch_all)

tcae_embedding=pd.DataFrame(tcae_embedding,index=tcae_all.index)
nn_embedding=pd.DataFrame(nn_embedding,index=nn_all.index)
np_embedding=pd.DataFrame(np_embedding,index=np_all.index)
nn_pit_embedding=pd.DataFrame(nn_pit_embedding,index=nn_pitch_all.index)
np_pit_embedding=pd.DataFrame(np_pit_embedding,index=np_pitch_all.index)


In [ ]:
# 형태 변환
tcae_test_audio = np.squeeze(tcae_embedding.loc[tcae_embedding.index.str.contains('보컬보컬'),:])
nn_test_audio = np.squeeze(nn_embedding.loc[nn_embedding.index.str.contains('보컬보컬'),:])
np_test_audio = np.squeeze(np_embedding.loc[np_embedding.index.str.contains('보컬보컬'),:])
nn_pit_test_audio = np.squeeze(nn_pit_embedding.loc[nn_pit_embedding.index.str.contains('보컬보컬'),:])
np_pit_test_audio = np.squeeze(np_pit_embedding.loc[np_pit_embedding.index.str.contains('보컬보컬'),:])

### 유사도 검증

In [ ]:
# 코사인
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))
    
# 피어슨
def pear_sim(a, b):
    return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [ ]:
def cos_sim_test(sim , embedding, test_audio):
  np_pear_lst = []
  name_lst = []
  for i in embedding.index:
    np_pear_lst.append(sim(test_audio, np.squeeze(embedding.loc[i,:])))
    name_lst.append(i)
      
  nn_pear_df = pd.DataFrame(np_pear_lst, name_lst)
  nn_pear_df.columns = ['cos']
  return nn_pear_df

cos_tcae = cos_sim_test(cos_sim, tcae_embedding,tcae_test_audio)
cos_nn = cos_sim_test(cos_sim, nn_embedding,nn_test_audio)
cos_np = cos_sim_test(cos_sim, np_embedding,np_test_audio)
cos_nn_pit = cos_sim_test(cos_sim ,nn_pit_embedding,nn_pit_test_audio)
cos_np_pit = cos_sim_test(cos_sim, np_pit_embedding,np_pit_test_audio)

pear_tcae = cos_sim_test(pear_sim, tcae_embedding,tcae_test_audio)
pear_nn = cos_sim_test(pear_sim, nn_embedding,nn_test_audio)
pear_np = cos_sim_test(pear_sim, np_embedding,np_test_audio)
pear_nn_pit = cos_sim_test(pear_sim ,nn_pit_embedding,nn_pit_test_audio)
pear_np_pit = cos_sim_test(pear_sim, np_pit_embedding,np_pit_test_audio)

cos_tcae['cos'] = cos_tcae['cos']*0.1 + cos_nn['cos']*0.07 + cos_np['cos']*0.08 + cos_nn_pit['cos']*0.12 + cos_np_pit['cos']*0.13+\
                  pear_tcae['cos']*0.1 + pear_nn['cos']*0.07 + pear_np['cos']*0.08 + pear_nn_pit['cos']*0.12 + pear_np_pit['cos']*0.13

print(cos_tcae.sort_values(by='cos', ascending = False).head(10))

### cf) 
- 차원축소 갯수 실험

In [ ]:
def song_test(data, sim, n_component, random_state, song_name, lamb=500):
    if n_component!=0:
        reducer = umap.UMAP(n_components=n_component, random_state = random_state)

    nn_embedding = reducer.fit_transform(data)
    nn_embedding=pd.DataFrame(nn_embedding, index = nn_extract.index)
    nn_test_audio = np.squeeze(nn_embedding.loc[nn_embedding.index.str.contains(song_name),:])

    nn_pear_lst = []
    name_lst = []

    for i in nn_embedding.index:
        if sim =='minkowski_sim':
            nn_pear_lst.append(sim(nn_test_audio, np.squeeze(nn_embedding.loc[i,:]),lamb))
        else:
            nn_pear_lst.append(sim(nn_test_audio, np.squeeze(nn_embedding.loc[i,:])))
        name_lst.append(i)

    nn_pear_df = pd.DataFrame(nn_pear_lst, name_lst)
    nn_pear_df.columns = ['pear']
    return nn_pear_df

- np, nn, pn 결합 실험

In [ ]:
np_pear = song_test(np_extract, pear_sim, 10, 0, '성시경_희재')
nn_pear = song_test(nn_extract, pear_sim, 10, 0, '성시경_희재')

In [ ]:
npnnpn_pear =  nn_pear*0.5 + np_pear*0.25 +pn_pear*0.25

In [ ]:
npnnpn_pear.sort_values(by='pear',ascending= False).head(10)

,pear
Final_dataset/성시경_희재.wav,1.000000
Final_dataset/규현_내 마음을 누르는 일 (Daystar).wav,0.998719
Final_dataset/임창정_그 곳에 멈춰서.wav,0.997331
Final_dataset/임창정_나눠갖지 말아요.wav,0.996976
Final_dataset/성시경_어디선가 언젠가.wav,0.996953
Final_dataset/성시경_사랑하는 일.wav,0.996387
Final_dataset/윤종신_뒷모습.wav,0.995995
Final_dataset/김조한_Love At Firstsight.wav,0.995985
Final_dataset/오왠_Good Night (Remastered).wav,0.995795
Final_dataset/나윤권_여름아 부탁해.wav,0.995753


https://goofcode.github.io/similarity-measure \
https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-4/07-similarity